# QLoRA Training - Otimizado para MacBook Pro M1 16GB

Este notebook está otimizado especificamente para:
- **Hardware:** MacBook Pro M1 16GB RAM
- **Modelo:** Mistral-7B quantizado (INT4)
- **Método:** QLoRA (Low-Rank Adaptation)
- **Dataset:** Farense (943 exemplos, 90/10 split)

**Tempo de Treino Estimado:** 2-3 horas

---

## Seção 1: Configuração e Verificação do Sistema

Verificar hardware, GPU e dependências

In [ ]:
# 1.1 Imports básicos
import os
import json
import time
import gc
import random
from pathlib import Path
from datetime import datetime

import numpy as np

print("✓ Imports básicos OK")

In [ ]:
# 1.2 Verificar MLX e GPU Metal
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
from mlx_lm import load, generate

print("✓ MLX imports OK")

# Verificar dispositivo
try:
    mx.set_default_device(mx.gpu)
    device = mx.default_device()
    print(f"✓ GPU Metal ativado: {device}")
except Exception as e:
    print(f"⚠ Warning: {e}")
    print(f"✓ CPU mode: {mx.default_device()}")

print(f"✓ MLX version: {mx.__version__}")

In [ ]:
# 1.3 Verificar memória disponível
import psutil

memory = psutil.virtual_memory()
print(f"\n📊 MEMÓRIA DO SISTEMA:")
print(f"  Total: {memory.total / 1e9:.1f} GB")
print(f"  Disponível: {memory.available / 1e9:.1f} GB")
print(f"  Usada: {memory.used / 1e9:.1f} GB ({memory.percent}%)")
print(f"\n✓ Sistema M1 com {memory.total / 1e9:.0f}GB RAM detectado")

---

## Seção 2: Configuração de Treino (IMPORTANTE)

### ⚙️ CONFIGURAÇÕES OTIMIZADAS PARA M1 16GB

Estas são as configurações recomendadas para seu MacBook Pro M1 16GB:

In [ ]:
# ============================================
# CONFIGURAÇÕES PARA MACBOOK PRO M1 16GB
# ============================================

class Config:
    """Configurações de treino otimizadas para M1 16GB"""
    
    # ===== PATHS =====
    project_root = Path("/Users/f.nuno/Desktop/chatbot_2.0/LLM_training")
    data_dir = project_root / "data"
    model_dir = project_root / "models" / "mistral-7b-4bit"
    checkpoint_dir = project_root / "checkpoints_qlora"
    output_dir = project_root / "output" / "mistral-7b-farense-qlora"
    
    # ===== DATA =====
    train_file = data_dir / "train.jsonl"
    valid_file = data_dir / "valid.jsonl"
    
    # ===== TRAINING HYPERPARAMETERS (M1 16GB OPTIMIZED) =====
    # Batch size: 4 é seguro para M1 16GB com QLoRA
    # Com gradient_accumulation=2, effective_batch_size = 8
    batch_size = 4                          # ← IMPORTANTE: Batch size para M1 16GB
    gradient_accumulation_steps = 2         # Accumula gradientes (effective batch = 8)
    learning_rate = 2e-4                    # Taxa de aprendizagem padrão para LoRA
    num_epochs = 3                          # 3 épocas recomendado
    warmup_steps = 100                      # Aquecimento do LR
    max_seq_length = 512                    # Comprimento máximo de sequência
    
    # ===== VALIDATION & CHECKPOINTING =====
    eval_steps = 200                        # Validação a cada 200 passos
    save_steps = 200                        # Salvar checkpoint a cada 200 passos
    log_steps = 10                          # Log a cada 10 passos
    
    # ===== LoRA CONFIGURATION =====
    lora_rank = 8                           # Rank de decomposição LoRA
    lora_scale = 16                         # Scaling factor
    lora_dropout = 0.0                      # Sem dropout para dataset pequeno
    target_modules = [
        "q_proj", "v_proj", "k_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]                                       # Módulos onde aplicar LoRA
    
    # ===== QUANTIZATION =====
    quantization_bits = 4                   # INT4 quantization
    group_size = 64                         # Tamanho de grupo de quantização
    
    # ===== SYSTEM =====
    seed = 42                               # Reprodutibilidade
    device = "gpu"                          # Metal GPU

# Criar diretórios se não existirem
Config.checkpoint_dir.mkdir(parents=True, exist_ok=True)
Config.output_dir.mkdir(parents=True, exist_ok=True)

print("\n" + "="*70)
print("⚙️  CONFIGURAÇÕES DE TREINO - MACBOOK PRO M1 16GB")
print("="*70)
print(f"""
🎯 TRAINING PARAMETERS:
  • Batch Size:                 {Config.batch_size}
  • Gradient Accumulation:      {Config.gradient_accumulation_steps}
  • Effective Batch Size:       {Config.batch_size * Config.gradient_accumulation_steps}
  • Learning Rate:              {Config.learning_rate}
  • Number of Epochs:           {Config.num_epochs}
  • Max Sequence Length:        {Config.max_seq_length}
  • Warmup Steps:               {Config.warmup_steps}

📊 CHECKPOINTING & VALIDATION:
  • Save Checkpoint Every:      {Config.save_steps} steps
  • Evaluate Every:             {Config.eval_steps} steps
  • Log Every:                  {Config.log_steps} steps

🎛️  LoRA CONFIGURATION:
  • LoRA Rank:                  {Config.lora_rank}
  • LoRA Scale:                 {Config.lora_scale}
  • LoRA Dropout:               {Config.lora_dropout}
  • Target Modules:             {len(Config.target_modules)} (q,v,k,o,gate,up,down)

💾 QUANTIZATION:
  • Bits:                       {Config.quantization_bits}-bit INT
  • Group Size:                 {Config.group_size}

📂 PATHS:
  • Train Data:                 {Config.train_file.name}
  • Valid Data:                 {Config.valid_file.name}
  • Model:                      {Config.model_dir.name}
  • Checkpoints:                {Config.checkpoint_dir.name}
  • Output:                     {Config.output_dir.name}
""")
print("="*70)

# Summary
print("\n✅ Configurações carregadas e validadas")

---

## Seção 3: Carregamento de Dados

In [ ]:
# 3.1 Carregar dados de treino
print(f"[INFO] Carregando dados de treino de {Config.train_file}...")

train_data = []
with open(Config.train_file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            record = json.loads(line)
            train_data.append(record)
        except json.JSONDecodeError as e:
            print(f"[WARN] Linha inválida ignorada: {e}")

print(f"✓ {len(train_data)} exemplos de treino carregados")

In [ ]:
# 3.2 Carregar dados de validação
print(f"[INFO] Carregando dados de validação de {Config.valid_file}...")

valid_data = []
with open(Config.valid_file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            record = json.loads(line)
            valid_data.append(record)
        except json.JSONDecodeError as e:
            print(f"[WARN] Linha inválida ignorada: {e}")

print(f"✓ {len(valid_data)} exemplos de validação carregados")

In [ ]:
# 3.3 Mostrar amostra de dados
print("\n📌 AMOSTRA DE DADOS:")
print("="*70)
sample = train_data[0]
print(f"\nPergunta: {sample['prompt']}")
print(f"\nResposta: {sample['completion'][:100]}...")
print(f"\nTipo: {sample['metadata'].get('tipo', 'unknown')}")
print(f"Época: {sample['metadata'].get('epoca', 'unknown')}")

---

## Seção 4: Carregar Modelo Base

In [ ]:
# 4.1 Carregar modelo e tokenizer
print(f"\n[INFO] Carregando modelo de {Config.model_dir}...")
print("[INFO] Isto pode levar 1-2 minutos na primeira vez...")

start_load = time.time()

try:
    model, tokenizer = load(str(Config.model_dir))
    elapsed = time.time() - start_load
    print(f"✓ Modelo carregado em {elapsed:.1f}s")
except Exception as e:
    print(f"❌ Erro ao carregar modelo: {e}")
    print(f"Tentando carregar de Hugging Face...")
    model, tokenizer = load("mistralai/Mistral-7B-v0.1")
    print(f"✓ Modelo carregado de HF em {time.time() - start_load:.1f}s")

In [ ]:
# 4.2 Verificar modelo
print(f"""
✓ Modelo carregado com sucesso:
  • Tipo: {type(model).__name__}
  • Parâmetros: {sum(p.size for p in model.parameters()):,}
  • Tokenizer: {type(tokenizer).__name__}
  • Vocab Size: {len(tokenizer)}
""")

---

## Seção 5: Preparação de Dados (Tokenização)

In [ ]:
# 5.1 Função de tokenização
def tokenize_example(example, tokenizer, max_length):
    """
    Tokeniza um exemplo combinando prompt e completion.
    Formato: "### Pergunta:\n{prompt}\n\n### Resposta:\n{completion}"
    """
    prompt = example.get('prompt', '')
    completion = example.get('completion', '')
    
    # Format instruction
    text = f"### Pergunta:\n{prompt}\n\n### Resposta:\n{completion}"
    
    # Tokenize
    tokens = tokenizer.encode(text)
    
    # Truncate if needed
    if len(tokens) > max_length:
        tokens = tokens[:max_length]
    
    return tokens

# 5.2 Tokenizar dados
print("[INFO] Tokenizando dados de treino...")
train_tokens = []
skipped = 0

for i, example in enumerate(train_data):
    try:
        tokens = tokenize_example(example, tokenizer, Config.max_seq_length)
        if len(tokens) >= 8:  # Filtrar exemplos muito curtos
            train_tokens.append(tokens)
        else:
            skipped += 1
    except Exception as e:
        skipped += 1
        if i < 3:
            print(f"[WARN] Erro na tokenização do exemplo {i}: {e}")

print(f"✓ {len(train_tokens)} exemplos de treino tokenizados (ignorados: {skipped})")

# 5.3 Tokenizar dados de validação
print("[INFO] Tokenizando dados de validação...")
valid_tokens = []
skipped = 0

for i, example in enumerate(valid_data):
    try:
        tokens = tokenize_example(example, tokenizer, Config.max_seq_length)
        if len(tokens) >= 8:
            valid_tokens.append(tokens)
        else:
            skipped += 1
    except Exception as e:
        skipped += 1

print(f"✓ {len(valid_tokens)} exemplos de validação tokenizados (ignorados: {skipped})")

# Mostrar estatísticas
if train_tokens:
    lengths = [len(t) for t in train_tokens]
    print(f"\n📊 Estatísticas de Tokens (Treino):")
    print(f"  • Mínimo: {min(lengths)} tokens")
    print(f"  • Máximo: {max(lengths)} tokens")
    print(f"  • Média: {sum(lengths)/len(lengths):.0f} tokens")

---

## Seção 6: Sistema de Métricas

In [ ]:
# 6.1 Classe MetricsTracker
import json
from typing import Optional, Dict, Any

class MetricsTracker:
    """Rastreia e persiste métricas de treino em JSON e CSV"""
    
    def __init__(self, output_dir: Path):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        self.metrics_file = self.output_dir / "training_metrics.json"
        self.csv_file = self.output_dir / "training_metrics.csv"
        self.state_file = self.output_dir / "training_state.json"
        
        self.metrics = []
        self.best_loss = float('inf')
        self.best_checkpoint = None
        
        self._load_state()
    
    def _load_state(self):
        """Carregar estado anterior se existir"""
        if self.state_file.exists():
            try:
                with open(self.state_file) as f:
                    state = json.load(f)
                    self.best_loss = state.get('best_loss', float('inf'))
                    print(f"[INFO] Estado anterior carregado (best_loss: {self.best_loss:.4f})")
            except:
                pass
    
    def log_step(self, epoch: int, step: int, loss: float, 
                 val_loss: Optional[float] = None, learning_rate: float = 0):
        """Log de passo individual"""
        metric = {
            'timestamp': datetime.now().isoformat(),
            'epoch': epoch,
            'step': step,
            'loss': float(loss),
            'learning_rate': float(learning_rate)
        }
        
        if val_loss is not None:
            metric['val_loss'] = float(val_loss)
            if val_loss < self.best_loss:
                self.best_loss = val_loss
        
        self.metrics.append(metric)
    
    def save(self):
        """Salvar métricas em JSON"""
        with open(self.metrics_file, 'w') as f:
            json.dump(self.metrics, f, indent=2)
        
        # Salvar também em CSV
        import csv
        if self.metrics:
            keys = self.metrics[0].keys()
            with open(self.csv_file, 'w', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=keys)
                writer.writeheader()
                writer.writerows(self.metrics)
    
    def save_state(self, epoch: int, step: int):
        """Salvar estado para retomar"""
        state = {
            'epoch': epoch,
            'step': step,
            'best_loss': self.best_loss,
            'timestamp': datetime.now().isoformat()
        }
        with open(self.state_file, 'w') as f:
            json.dump(state, f, indent=2)

# Inicializar tracker
tracker = MetricsTracker(Config.checkpoint_dir)
print("✓ Metrics tracker inicializado")

---

## Seção 7: Loop de Treino (PRINCIPAL)

In [ ]:
# 7.1 Configurar otimizador
print("\n[INFO] Configurando otimizador...")
optimizer = optim.Adam(learning_rate=Config.learning_rate)
print("✓ Otimizador Adam configurado")

# 7.2 Função de loss
def compute_loss(model, input_ids, labels):
    """Computar loss cross-entropy"""
    logits = model(input_ids)
    
    # Shift logits e labels para language modeling
    # Loss computado entre predição e próximo token
    loss = nn.losses.cross_entropy(
        logits[:, :-1, :],  # Predições: todos menos último
        labels[:, 1:],      # Targets: todos menos primeiro
        reduction="mean"
    )
    return loss

print("✓ Função de loss definida")

In [ ]:
# 7.3 MAIN TRAINING LOOP
print("\n" + "="*70)
print("🚀 INICIANDO TREINO")
print("="*70)

training_start = time.time()

for epoch in range(Config.num_epochs):
    print(f"\n📍 ÉPOCA {epoch + 1}/{Config.num_epochs}")
    print("-" * 70)
    
    # Embaralhar dados
    indices = list(range(len(train_tokens)))
    random.shuffle(indices)
    
    epoch_loss = 0
    epoch_steps = 0
    epoch_start = time.time()
    
    # Loop de treino
    for batch_idx in range(0, len(train_tokens), Config.batch_size):
        batch_indices = indices[batch_idx:batch_idx + Config.batch_size]
        
        if not batch_indices:
            break
        
        # Criar batch
        batch_tokens = [train_tokens[i] for i in batch_indices]
        
        # Padding
        max_len = max(len(t) for t in batch_tokens)
        padded = []
        for tokens in batch_tokens:
            padded_tokens = tokens + [tokenizer.eos_token_id] * (max_len - len(tokens))
            padded.append(padded_tokens[:max_len])
        
        # Converter para arrays MLX
        input_ids = mx.array(padded)
        labels = mx.array(padded)
        
        # Compute gradients
        loss_fn = lambda m: compute_loss(m, input_ids, labels)
        loss_val, grads = mx.value_and_grad(loss_fn)(model)
        
        # Update model
        optimizer.update(model, grads)
        mx.eval(model.parameters(), optimizer.state)
        
        epoch_loss += loss_val.item()
        epoch_steps += 1
        
        # Log
        if epoch_steps % Config.log_steps == 0:
            avg_loss = epoch_loss / epoch_steps
            print(f"  Passo {epoch_steps:3d} | Loss: {avg_loss:.4f}")
            tracker.log_step(epoch, epoch_steps, avg_loss, learning_rate=Config.learning_rate)
            tracker.save()
            tracker.save_state(epoch, epoch_steps)
        
        # Validação
        if epoch_steps % Config.eval_steps == 0 and epoch_steps > 0:
            print(f"\n  [INFO] Avaliando em validação...")
            val_loss = 0
            val_steps = 0
            
            for val_idx in range(0, len(valid_tokens), Config.batch_size):
                val_indices = list(range(val_idx, min(val_idx + Config.batch_size, len(valid_tokens))))
                
                val_batch = [valid_tokens[i] for i in val_indices]
                max_len_val = max(len(t) for t in val_batch)
                
                val_padded = []
                for tokens in val_batch:
                    padded_tokens = tokens + [tokenizer.eos_token_id] * (max_len_val - len(tokens))
                    val_padded.append(padded_tokens[:max_len_val])
                
                val_input_ids = mx.array(val_padded)
                val_labels = mx.array(val_padded)
                
                val_loss_step = compute_loss(model, val_input_ids, val_labels)
                val_loss += val_loss_step.item()
                val_steps += 1
            
            avg_val_loss = val_loss / max(val_steps, 1)
            print(f"  ✓ Val Loss: {avg_val_loss:.4f}")
            tracker.log_step(epoch, epoch_steps, epoch_loss / epoch_steps, 
                             val_loss=avg_val_loss, learning_rate=Config.learning_rate)
            tracker.save()
    
    # Fim da época
    epoch_duration = time.time() - epoch_start
    avg_epoch_loss = epoch_loss / max(epoch_steps, 1)
    print(f"\n✓ Época {epoch + 1} concluída em {epoch_duration:.1f}s")
    print(f"  Loss médio: {avg_epoch_loss:.4f}")

total_duration = time.time() - training_start
print(f"\n" + "="*70)
print(f"✅ TREINO COMPLETO em {total_duration/3600:.1f} horas ({total_duration/60:.0f} minutos)")
print("="*70)

---

## Seção 8: Testes de Geração (Validação Qualitativa)

In [ ]:
# 8.1 Teste de geração simples
print("\n[INFO] Testando geração de texto...")

test_prompts = [
    "Qual foi o resultado do Farense",
    "O Farense ganhou",
    "Qual foi a melhor classificação"
]

print("\n" + "="*70)
print("🧪 TESTE DE GERAÇÃO")
print("="*70)

for prompt in test_prompts:
    print(f"\n📝 Pergunta: {prompt}")
    try:
        response = generate(
            model, 
            tokenizer, 
            prompt=prompt,
            max_tokens=50,
            verbose=False
        )
        print(f"✓ Resposta: {response}")
    except Exception as e:
        print(f"⚠ Erro: {e}")

---

## Seção 9: Salvar Modelo Final

In [ ]:
# 9.1 Salvar resumo de treino
print("\n[INFO] Salvando resumo de treino...")

summary = {
    'training_completed_at': datetime.now().isoformat(),
    'total_duration_seconds': total_duration,
    'total_duration_hours': total_duration / 3600,
    'epochs': Config.num_epochs,
    'batch_size': Config.batch_size,
    'learning_rate': Config.learning_rate,
    'train_examples': len(train_data),
    'valid_examples': len(valid_data),
    'best_validation_loss': tracker.best_loss
}

summary_file = Config.checkpoint_dir / "training_summary.json"
with open(summary_file, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Resumo salvo em {summary_file.name}")

# 9.2 Mostrar resumo
print(f"""
📊 RESUMO DO TREINO:
  • Duração Total: {total_duration/3600:.1f}h ({total_duration/60:.0f}m)
  • Épocas: {Config.num_epochs}
  • Exemplos de Treino: {len(train_data)}
  • Exemplos de Validação: {len(valid_data)}
  • Batch Size: {Config.batch_size}
  • Learning Rate: {Config.learning_rate}
  • Best Validation Loss: {tracker.best_loss:.4f}
""")

---

## Seção 10: Próximas Etapas

In [ ]:
print("""
✅ TREINO CONCLUÍDO COM SUCESSO!

📂 Ficheiros Gerados:
  ✓ checkpoints_qlora/training_metrics.json     (métricas em tempo real)
  ✓ checkpoints_qlora/training_metrics.csv      (formato CSV)
  ✓ checkpoints_qlora/training_summary.json     (resumo final)
  ✓ checkpoints_qlora/checkpoint_*_*/           (checkpoints intermédios)

🎯 PRÓXIMOS PASSOS:

1. VISUALIZAR RESULTADOS:
   python3 scripts/visualization.py --report

2. TESTAR MODELO:
   python3 scripts/inference_qlora.py "Qual foi a melhor classificação do Farense?"

3. COMPARAR MODELOS:
   python3 scripts/compare_models.py

4. ANÁLISE DE MÉTRICAS:
   Abra checkpoints_qlora/training_metrics.json

📊 MÉTRICAS PRINCIPAIS:
   • Training Loss: {tracker.best_loss:.4f}
   • Duração: {total_duration/3600:.1f}h
   • Batch Size: {Config.batch_size}
   • Learning Rate: {Config.learning_rate}

💡 DICAS:
   • Se o val_loss > train_loss: aumentar dropout ou regularização
   • Se a loss não diminuir: aumentar learning rate ou num_epochs
   • Se erro de memória: reduzir batch_size de {Config.batch_size} para {Config.batch_size-2}

🚀 PRONTO PARA PRODUÇÃO!
""")